<a href="https://colab.research.google.com/github/lazzarusd/Coursera_Capstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Find the best place to open a Restaurant
## In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it?

### To try to answer this question, I decided to focus my attention to Milan, the most important city in Italy for business and startup activities.

#### First step, let's install the python packages 

In [ ]:
!pip install wikipedia
!pip install folium
!pip install geopy
!pip install geopandas

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=79d768f558f66ceeac56ced602addaf57ba6b7c67a14a0fc1de59db2e99c336d
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 931kB 2.7MB/s 
     |████████████████████████████████| 14.7MB 293kB/s 
     |████████████████████████████████| 10.9MB 34.1MB/s 


#### and import some useful libraries

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import wikipedia as wp
import requests
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopandas as gpd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap

print('Libraries imported.')

Libraries imported.


#### First we need some information about the area of Milan such as borough, districts, population etc... I think a good place to take a look is wikipedia

In [ ]:
wp.set_lang("it")
html = wp.page("Municipi_di_Milano").html()

milan_borough_wiki = pd.read_html(html)[3]

milan_borough_wiki['#'] = milan_borough_wiki['#'].str.replace('Municipio ', '')

milan_borough_wiki.columns = ('Borough', 'Name', 'Area(km2)', 'Population(31/12/2018)', 'Population_Density(km2)', 'Neighborhood', 'Localizzazione')

milan_borough_wiki.drop(columns=["Localizzazione"], inplace=True)
milan_borough_wiki.drop(columns=["Neighborhood"], inplace=True)
milan_borough_wiki.drop(milan_borough_wiki.tail(1).index, inplace=True)

# milan_borough.set_index('Borough', inplace=True)
milan_borough_wiki

,Borough,Name,Area(km2),Population(31/12/2018),Population_Density(km2)
0,1,Centro storico,967,98 531,10 189
1,2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884
2,3,"Città Studi, Lambrate, Venezia",1423,144 110,10 127
3,4,"Vittoria, Forlanini",2095,161 551,7 711
4,5,"Vigentino, Chiaravalle, Gratosoglio",2987,126 089,4 221
5,6,"Barona, Lorenteggio",1828,151 291,8 276
6,7,"Baggio, De Angeli, San Siro",3134,175 465,5 598
7,8,"Fiera, Gallaratese, Quarto Oggiaro",2372,188 367,7 941
8,9,"Stazione Garibaldi, Niguarda",2112,187 773,8 890


#### Now let's clean the dataset and add the latitude and longitude of each districts 

In [ ]:
geo_list = []

for x in range(1,10):
    page = requests.get('https://en.wikipedia.org/wiki/Zone_' + str(x) + '_of_Milan')
    el = BeautifulSoup(page.content, 'html.parser').find(id='coordinates').find('span', class_='geo')
    geo_list.append(el.text.split("; "))

lat, lon = zip(*geo_list)

milan_borough_wiki['Latitude'] = lat
milan_borough_wiki['Longitude'] = lon
milan_borough_wiki

,Borough,Name,Area(km2),Population(31/12/2018),Population_Density(km2),Latitude,Longitude
0,1,Centro storico,967,98 531,10 189,45.471282,9.184999
1,2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
2,3,"Città Studi, Lambrate, Venezia",1423,144 110,10 127,45.482506,9.241047
3,4,"Vittoria, Forlanini",2095,161 551,7 711,45.431573,9.244738
4,5,"Vigentino, Chiaravalle, Gratosoglio",2987,126 089,4 221,45.416987,9.238333
5,6,"Barona, Lorenteggio",1828,151 291,8 276,45.440087,9.155924
6,7,"Baggio, De Angeli, San Siro",3134,175 465,5 598,45.461244,9.089917
7,8,"Fiera, Gallaratese, Quarto Oggiaro",2372,188 367,7 941,45.515925,9.140196
8,9,"Stazione Garibaldi, Niguarda",2112,187 773,8 890,45.516888,9.191866


#### Download and prepare the dataset of Milan's neighborhood
Source: 

*   https://www.pgt.comune.milano.it/sites/default/files/allegati/NIL_Intro.pdf for the link district\neighborhood    
(In wikipedia all the info aren't up to date)
*   https://www.mercato-immobiliare.info/lombardia/milano/milano.html and      
https://www.immobiliare.it/mercato-immobiliare/lombardia/milano/ for the average land price



In [ ]:
url = 'https://raw.githubusercontent.com/lazzarusd/Coursera_Capstone/master/file/Milano_Municipi_NIL.csv'
milan_borough = pd.read_csv(url)
milan_borough.rename(columns={"Num_NIL": "Num_Neighborhood", "NIL": "Neighborhood", "Municipio": "Borough", "prezzo_mq": "Average_price_sm"}, inplace=True)
milan_borough.head()

,Num_Neighborhood,Neighborhood,Borough,Average_price_sm
0,17,Adriano,2,€ 2.800 /m²
1,80,Affori,9,€ 2.350 /m²
2,87,Assiano,7,€ 2.400 /m²
3,55,Baggio - Q.re degli Olmi - Q.re Valsesia,7,€ 2.400 /m²
4,52,Bande Nere,6,€ 3.857 /m²


#### Merge with wikipedia dataset 

In [ ]:
milan_borough['Borough'] = milan_borough['Borough'].astype(int)
milan_borough_wiki['Borough'] = milan_borough_wiki['Borough'].astype(int)

milan_borough = pd.merge(milan_borough, milan_borough_wiki, on='Borough')
milan_borough.head()

,Num_Neighborhood,Neighborhood,Borough,Average_price_sm,Name,Area(km2),Population(31/12/2018),Population_Density(km2),Latitude,Longitude
0,17,Adriano,2,€ 2.800 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
1,16,Gorla - Precotto,2,€ 2.800 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
2,19,Padova - Turro - Crescenzago,2,€ 2.900 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
3,10,Stazione Centrale - Ponte Seveso,2,€ 4.950 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
4,80,Affori,9,€ 2.350 /m²,"Stazione Garibaldi, Niguarda",2112,187 773,8 890,45.516888,9.191866


#### Fortunately the statistics office of Milan create a very interesting portal about open data: https://dati.comune.milano.it/.   
#### So I used it to get the shape file of Milan's neighborhood


In [ ]:
milan_neighborhood_geodf = gpd.read_file('https://dati.comune.milano.it/dataset/e8e765fc-d882-40b8-95d8-16ff3d39eb7c/resource/9c4e0776-56fc-4f3d-8a90-f4992a3be426/download/ds964_nil_wm.geojson')
milan_neighborhood_geodf.head()

,ID_NIL,NIL,Valido_dal,Valido_al,Fonte,Shape_Length,Shape_Area,OBJECTID,geometry
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,8723.368714,2.406306e+06,89,"POLYGON ((9.15422 45.43775, 9.15274 45.43887, ..."
1,64,TRENNO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3309.998800,4.896921e+05,90,"POLYGON ((9.10623 45.49016, 9.10591 45.49084, ..."
2,67,PORTELLO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3800.750663,9.096022e+05,91,"POLYGON ((9.15636 45.48785, 9.15495 45.48852, ..."
3,81,BOVISASCA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,7105.469715,1.578028e+06,92,"POLYGON ((9.16803 45.52234, 9.16763 45.52272, ..."
4,84,PARCO NORD,05/02/2020,Vigente,Milano 2030 - PGT Approvato,11741.717005,1.532331e+06,93,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."


In [ ]:
milan_neighborhood = milan_neighborhood_geodf[['ID_NIL', 'NIL']].copy()
milan_neighborhood.columns = ('Id', 'Neighborhood')
milan_neighborhood['Neighborhood'] = milan_neighborhood['Neighborhood'].str.title()
milan_neighborhood.tail()

,Id,Neighborhood
83,68,Pagano
84,2,Brera
85,33,Rogoredo - Santa Giulia
86,17,Adriano
87,53,Lorenteggio


In [ ]:
from shapely.wkt import loads as load_wkt

centroid_list = []

for polygon in milan_neighborhood_geodf["geometry"]:
    box_str = str(polygon)
    p1 = load_wkt(box_str)
    point = p1.centroid
#     print(type(p1.centroid.x))
#     print(p1.centroid.y)
    centroid_list.append((p1.centroid.y, p1.centroid.x))

lat_centr, lon_centr = zip(*centroid_list)

milan_neighborhood['Latitude'] = lat_centr
milan_neighborhood['Longitude'] = lon_centr
milan_neighborhood.head()

,Id,Neighborhood,Latitude,Longitude
0,48,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.438460,9.137260
1,64,Trenno,45.492822,9.101675
2,67,Portello,45.484490,9.153947
3,81,Bovisasca,45.517433,9.156731
4,84,Parco Nord,45.523514,9.184235


In [ ]:
milan_borough.head()

,Num_Neighborhood,Neighborhood,Borough,Average_price_sm,Name,Area(km2),Population(31/12/2018),Population_Density(km2),Latitude,Longitude
0,17,Adriano,2,€ 2.800 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
1,16,Gorla - Precotto,2,€ 2.800 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
2,19,Padova - Turro - Crescenzago,2,€ 2.900 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
3,10,Stazione Centrale - Ponte Seveso,2,€ 4.950 /m²,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884,45.486117,9.203635
4,80,Affori,9,€ 2.350 /m²,"Stazione Garibaldi, Niguarda",2112,187 773,8 890,45.516888,9.191866


#### Let's create the final input dataset 

In [ ]:
new_list = []

for ind1, neighborhood in milan_neighborhood.iterrows():
    for ind2, borough in milan_borough.iterrows():
        s1 = neighborhood[["Neighborhood"]].item().lower().partition(' - ')[0]
        s2 = borough[["Neighborhood"]].item().lower()
#         print(str(type(s1)) + ": " + s1 + " # " + str(type(s2)) + ": " + s2)
        if s1 in s2:
            frame = {'Id': neighborhood[["Id"]].item(), 
                     'Neighborhood': neighborhood[["Neighborhood"]].item(),
                     'Borough': borough[["Borough"]].item(),
                     'Population(31/12/2018) Borough': borough[["Population(31/12/2018)"]].item(),
                     'Average Price(€/sm)': borough[["Average_price_sm"]].item(),
                     'Latitude': neighborhood[["Latitude"]].item(),
                     'Longitude': neighborhood[["Longitude"]].item()
                    }
            new_list.append(frame)

cn = ['Id', 'Neighborhood', 'Borough', 'Population(31/12/2018) Borough', 'Average Price(€/sm)', 'Latitude', 'Longitude']
df_milan_complete = pd.DataFrame(new_list, columns = cn)
df_milan_complete.head()

,Id,Neighborhood,Borough,Population(31/12/2018) Borough,Average Price(€/sm),Latitude,Longitude
0,48,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,6,151 291,€ 2.563 /m²,45.438460,9.137260
1,64,Trenno,8,188 367,€ 2.350 /m²,45.492822,9.101675
2,67,Portello,8,188 367,€ 4.300 /m²,45.484490,9.153947
3,81,Bovisasca,9,187 773,€ 2.000 /m²,45.517433,9.156731
4,84,Parco Nord,9,187 773,€ 6.800 /m²,45.523514,9.184235


### Create map of Milan using latitude and longitude values 

In [ ]:
address = 'Milan, MI'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan are 45.4668, 9.1905.


#### It's pretty important to use some good visualization to understand better the area 

In [ ]:
map_milan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_milan_complete['Latitude'], df_milan_complete['Longitude'], df_milan_complete['Id'], df_milan_complete['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milan)  
    
map_milan

## Now it's time to use the foursquare API to examinate the city of Milan

In [ ]:
#@title

CLIENT_ID = 'CPAC2GRN3VCADF3FNWJAPLQYMNICM0YBUAGHFXMZEGRZXDPG' # your Foursquare ID
CLIENT_SECRET = 'LOH4RGLND40CSDX10KBUTD2OV3GQN3DJXCTIPO54DTNDI4UW' # your Foursquare Secret
VERSION = '20200605'
LIMIT = 100

### Extract venues for each neighborhood in Milan

In [ ]:
def getNearbyRestaurants(names, latitudes, longitudes, radius=3000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        resp = requests.get(url).json()["response"]
        
        if "groups" in resp:
            results = resp['groups'][0]['items']
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
milan_restaurants = getNearbyRestaurants(names=df_milan_complete['Neighborhood'],
                                         latitudes=df_milan_complete['Latitude'],
                                         longitudes=df_milan_complete['Longitude'])

Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro
Trenno
Portello
Bovisasca
Parco Nord
Figino
Quarto Oggiaro - Vialba - Musocco
Isola
Quarto Cagnino
Stadio - Ippodromi
Quinto Romano
Duomo
Guastalla
San Siro
Comasina
Tibaldi
De Angeli - Monte Rosa
Farini
Bruzzano
Stephenson
Cantalupa
Quintosole
Parco Sempione
Barona
Villapizzone - Cagnola - Boldinasco
Gorla - Precotto
Niguarda - Ca' Granda - Prato Centenaro - Q.Re Fulvio Testi
Triulzo Superiore
Taliedo - Morsenchio - Q.Re Forlanini
Porta Ticinese - Conca Del Naviglio
Porta Ticinese - Conca Del Naviglio
Tre Torri
Assiano
Morivione
Vigentino - Q.Re Fatima
Bicocca
Ortomercato
Lodi - Corvetto
Muggiano
Porta Ticinese - Conchetta
Porta Ticinese - Conchetta
Umbria - Molise - Calvairate
Roserio
Ronchetto Delle Rane
Q.Re Gallaratese - Q.Re San Leonardo - Lampugnano
Padova - Turro - Crescenzago
Gratosoglio - Q.Re Missaglia - Q.Re Terrazze
Porta Magenta
Forze Armate
Ghisolfa
Chiaravalle
Parco Delle Abbazie
Bande Nere
Lambrate - Ortica
Stadera - Chies

Ok so each row is the dataset is restaurant opened in Milan

In [ ]:
print(milan_restaurants.shape)
milan_restaurants.head()

(7043, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.43846,9.13726,Osteria Ma.Si.,45.444339,9.140705,Italian Restaurant
1,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.43846,9.13726,Mio,45.442099,9.132717,Italian Restaurant
2,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.43846,9.13726,Forno Barona,45.435601,9.151299,Bakery
3,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.43846,9.13726,Arepa'z,45.447194,9.158641,South American Restaurant
4,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.43846,9.13726,Anatolia Döner Kebap,45.451472,9.151342,Kebab Restaurant


Remove the duplicates

In [ ]:
milan_restaurants_unique = milan_restaurants.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude'], keep='first')

and check if we can find in Milan vegan restaurant 

In [ ]:
milan_restaurants_unique[milan_restaurants_unique['Venue Category'].str.contains('Vegan')].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
288,Bovisasca,45.517433,9.156731,Misunchi,45.503580,9.162297,Vegetarian / Vegan Restaurant
699,Isola,45.490894,9.189617,Joia,45.477134,9.201806,Vegetarian / Vegan Restaurant
997,Duomo,45.463707,9.186948,Mudra Vegamore,45.453253,9.179246,Vegetarian / Vegan Restaurant
2192,Villapizzone - Cagnola - Boldinasco,45.497419,9.143523,Mens@sana,45.490061,9.164685,Vegetarian / Vegan Restaurant
2414,Triulzo Superiore,45.427941,9.249243,Arcobaleno Vegetariano,45.439293,9.224843,Vegetarian / Vegan Restaurant


For the last check, I want to see all the place in the map.

In [ ]:
rest_map = folium.Map([latitude, longitude], zoom_start=12)

for label, lat, lng in zip(milan_restaurants_unique['Venue'], 
                           milan_restaurants_unique['Venue Latitude'], 
                           milan_restaurants_unique['Venue Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        colur='red',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(rest_map)

from IPython.display import display
display(rest_map)

In [ ]:
milan_restaurants_unique.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adriano,23,23,23,23,23,23
Assiano,38,38,38,38,38,38
Bande Nere,2,2,2,2,2,2
Barona,9,9,9,9,9,9
Bicocca,16,16,16,16,16,16
Bovisasca,71,71,71,71,71,71
Bruzzano,12,12,12,12,12,12
Cantalupa,56,56,56,56,56,56
Cimiano - Rottole - Q.Re Feltre,26,26,26,26,26,26


Let's group by neighborhood all the new info get by foursquare

In [ ]:
milan_onehot = pd.get_dummies(milan_restaurants_unique['Venue Category'])
milan_onehot.insert(loc=0, column='Neighborhood', value=milan_restaurants_unique['Neighborhood'])
milan_grouped = milan_onehot.groupby('Neighborhood').mean().reset_index()
milan_grouped.head()

,Neighborhood,Abruzzo Restaurant,African Restaurant,Agriturismo,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Cafeteria,Café,Calabria Restaurant,Campanian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lombard Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Noodle House,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Piadineria,Pizza Place,Puglia Restaurant,Ramen Restaurant,Restaurant,Roman Restaurant,Romanian Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Sardinian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Sicilian Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.043478,0.0,0.000000,0.0,0.043478,0.0,0.000000,0.000000,0.217391,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.000000,0.0,0.043478,0.000000,0.043478,0.0,0.000000,0.043478,0.0,0.0,0.0,0.0,0.0,0.043478,0.086957,0.086957,0.0,0.0,0.0,0.000000,0.043478,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.217391,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0
1,Assiano,0.0,0.0,0.0,0.0,0.0,0.026316,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.026316,0.026316,0.131579,0.0,0.0,0.0,0.0,0.026316,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.078947,0.0,0.000000,0.026316,0.000000,0.0,0.026316,0.078947,0.0,0.0,0.0,0.0,0.0,0.131579,0.026316,0.000000,0.0,0.0,0.0,0.026316,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.026316,0.157895,0.0,0.0,0.078947,0.0,0.0,0.0,0.0,0.0,0.0,0.026316,0.026316,0.0,0.0,0.026316,0.0000,0.0,0.0,0.0,0.026316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Barona,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.111111,0.0,0.000000,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.222222,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.111111,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Bicocca,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.062500,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.062500,0.125000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.062500,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,0.062500,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0,0.0,0.062500,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,

This function return the most common venues for each row

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
milan_venues_sorted = pd.DataFrame(columns=columns)
milan_venues_sorted['Neighborhood'] = milan_grouped['Neighborhood']

for ind in np.arange(milan_grouped.shape[0]):
    milan_venues_sorted.iloc[ind, 1:] = return_most_common_venues(milan_grouped.iloc[ind, :], num_top_venues)

### Milan top 10 for each neighborhoods

In [ ]:
milan_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adriano,Pizza Place,Café,Japanese Restaurant,Kebab Restaurant,Vegetarian / Vegan Restaurant,Diner,Gastropub,Food Truck,Bakery,Mexican Restaurant
1,Assiano,Pizza Place,Italian Restaurant,Café,Fast Food Restaurant,Restaurant,Gastropub,Asian Restaurant,Burger Joint,Fried Chicken Joint,Japanese Restaurant
2,Bande Nere,Italian Restaurant,Café,Creperie,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant
3,Barona,Italian Restaurant,Steakhouse,Pizza Place,Café,Middle Eastern Restaurant,Bistro,Restaurant,Japanese Restaurant,Food Truck,Dim Sum Restaurant
4,Bicocca,Italian Restaurant,Pizza Place,Café,Bakery,Cafeteria,South American Restaurant,Fast Food Restaurant,Restaurant,Japanese Restaurant,Food Truck
5,Bovisasca,Italian Restaurant,Pizza Place,Café,Bakery,Diner,Restaurant,Japanese Restaurant,Kebab Restaurant,Chinese Restaurant,Doner Restaurant
6,Bruzzano,Pizza Place,Café,Italian Restaurant,Brazilian Restaurant,Breakfast Spot,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant
7,Cantalupa,Italian Restaurant,Pizza Place,Café,Restaurant,Japanese Restaurant,Fast Food Restaurant,Kebab Restaurant,Diner,Chinese Restaurant,Steakhouse
8,Cimiano - Rottole - Q.Re Feltre,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Sushi Restaurant,Food Court,Fast Food Restaurant,Bakery,Indian Restaurant,Asian Restaurant
9,Comasina,Café,Pizza Place,Restaurant,Food Court,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Piadineria,Diner


# Clustering
Now I will use the k-means algorithm to try to find some cluster that it can help me to classify the resturants in Milan

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

milan_grouped_clustering = milan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 4, 2, 4, 4, 4, 1, 4, 4, 0], dtype=int32)

In [ ]:
# add clustering labels
milan_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

milan_merged = df_milan_complete

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
milan_merged = milan_merged.join(milan_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
milan_merged.head() # check the last columns!

,Id,Neighborhood,Borough,Population(31/12/2018) Borough,Average Price(€/sm),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,6,151 291,€ 2.563 /m²,45.438460,9.137260,4.0,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Restaurant,Bakery,Sushi Restaurant,Chinese Restaurant,Mediterranean Restaurant,Steakhouse
1,64,Trenno,8,188 367,€ 2.350 /m²,45.492822,9.101675,4.0,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Burger Joint,Asian Restaurant
2,67,Portello,8,188 367,€ 4.300 /m²,45.484490,9.153947,4.0,Italian Restaurant,Pizza Place,Japanese Restaurant,Café,Restaurant,Sushi Restaurant,Sandwich Place,Bakery,Bistro,Seafood Restaurant
3,81,Bovisasca,9,187 773,€ 2.000 /m²,45.517433,9.156731,4.0,Italian Restaurant,Pizza Place,Café,Bakery,Diner,Restaurant,Japanese Restaurant,Kebab Restaurant,Chinese Restaurant,Doner Restaurant
4,84,Parco Nord,9,187 773,€ 6.800 /m²,45.523514,9.184235,1.0,Pizza Place,Italian Restaurant,Café,Seafood Restaurant,Japanese Restaurant,Piadineria,Restaurant,Sandwich Place,Gastropub,Sushi Restaurant


In [ ]:
milan_merged['Cluster Labels'] = milan_merged['Cluster Labels'].fillna(0)
milan_merged['Cluster Labels'] = milan_merged['Cluster Labels'].astype(int)
milan_merged.head()

,Id,Neighborhood,Borough,Population(31/12/2018) Borough,Average Price(€/sm),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,6,151 291,€ 2.563 /m²,45.438460,9.137260,4,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Restaurant,Bakery,Sushi Restaurant,Chinese Restaurant,Mediterranean Restaurant,Steakhouse
1,64,Trenno,8,188 367,€ 2.350 /m²,45.492822,9.101675,4,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Burger Joint,Asian Restaurant
2,67,Portello,8,188 367,€ 4.300 /m²,45.484490,9.153947,4,Italian Restaurant,Pizza Place,Japanese Restaurant,Café,Restaurant,Sushi Restaurant,Sandwich Place,Bakery,Bistro,Seafood Restaurant
3,81,Bovisasca,9,187 773,€ 2.000 /m²,45.517433,9.156731,4,Italian Restaurant,Pizza Place,Café,Bakery,Diner,Restaurant,Japanese Restaurant,Kebab Restaurant,Chinese Restaurant,Doner Restaurant
4,84,Parco Nord,9,187 773,€ 6.800 /m²,45.523514,9.184235,1,Pizza Place,Italian Restaurant,Café,Seafood Restaurant,Japanese Restaurant,Piadineria,Restaurant,Sandwich Place,Gastropub,Sushi Restaurant


##Check the final result on the folium map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milan_merged['Latitude'], milan_merged['Longitude'], milan_merged['Neighborhood'], milan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Let's take a look on the clusters

In [ ]:
milan_merged.loc[milan_merged['Cluster Labels'] == 0, milan_merged.columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Comasina,45.526441,9.159969,0,Café,Pizza Place,Restaurant,Food Court,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Piadineria,Diner
16,De Angeli - Monte Rosa,45.474878,9.148412,0,Bistro,Kebab Restaurant,Café,BBQ Joint,Food,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant
19,Stephenson,45.512246,9.121394,0,Café,Italian Restaurant,Sandwich Place,Food Truck,Sushi Restaurant,Food Court,Falafel Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
26,Niguarda - Ca' Granda - Prato Centenaro - Q.Re...,45.516696,9.196117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Tre Torri,45.476951,9.155728,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Muggiano,45.451403,9.071630,0,Café,Pizza Place,Italian Restaurant,Burger Joint,Fast Food Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant
44,Q.Re Gallaratese - Q.Re San Leonardo - Lampugnano,45.496596,9.111701,0,Bistro,Café,Food,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant
46,Gratosoglio - Q.Re Missaglia - Q.Re Terrazze,45.411688,9.171188,0,Café,Trattoria/Osteria,Pizza Place,Wings Joint,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant
49,Ghisolfa,45.491002,9.162986,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Chiaravalle,45.416749,9.239611,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
milan_merged.loc[milan_merged['Cluster Labels'] == 1, milan_merged.columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Parco Nord,45.523514,9.184235,1,Pizza Place,Italian Restaurant,Café,Seafood Restaurant,Japanese Restaurant,Piadineria,Restaurant,Sandwich Place,Gastropub,Sushi Restaurant
8,Quarto Cagnino,45.473740,9.108096,1,Pizza Place,Café,Italian Restaurant,Chinese Restaurant,Japanese Restaurant,Food Court,Restaurant,Kebab Restaurant,Sushi Restaurant,Food
10,Quinto Romano,45.479418,9.087541,1,Café,Food Truck,Pizza Place,Italian Restaurant,Japanese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant
18,Bruzzano,45.529177,9.172134,1,Pizza Place,Café,Italian Restaurant,Brazilian Restaurant,Breakfast Spot,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant
27,Triulzo Superiore,45.427941,9.249243,1,Pizza Place,Café,Italian Restaurant,Seafood Restaurant,Restaurant,Bakery,Sandwich Place,Middle Eastern Restaurant,Food Court,Moroccan Restaurant
33,Morivione,45.440972,9.193815,1,Greek Restaurant,Café,Mexican Restaurant,Pizza Place,Wings Joint,Fast Food Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant
47,Porta Magenta,45.462063,9.156051,1,Chinese Restaurant,Pizza Place,Irish Pub,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant
48,Forze Armate,45.462489,9.113830,1,Japanese Restaurant,Pizza Place,Italian Restaurant,Calabria Restaurant,Mediterranean Restaurant,Noodle House,Filipino Restaurant,Diner,Doner Restaurant,Eastern European Restaurant
51,Parco Delle Abbazie,45.411618,9.205639,1,Café,Pizza Place,Italian Restaurant,Kebab Restaurant,Filipino Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant
54,Stadera - Chiesa Rossa - Q.Re Torretta - Conca...,45.429940,9.178438,1,Pizza Place,Café,Italian Restaurant,Japanese Restaurant,Bistro,Gastropub,Restaurant,Hot Dog Joint,Asian Restaurant,Food Court


In [ ]:
milan_merged.loc[milan_merged['Cluster Labels'] == 2, milan_merged.columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Parco Sempione,45.474131,9.176251,2,Italian Restaurant,Restaurant,Irish Pub,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant
29,Porta Ticinese - Conca Del Naviglio,45.450475,9.181311,2,Italian Restaurant,Southern / Soul Food Restaurant,Café,Sushi Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant
30,Porta Ticinese - Conca Del Naviglio,45.450475,9.181311,2,Italian Restaurant,Southern / Soul Food Restaurant,Café,Sushi Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant
41,Umbria - Molise - Calvairate,45.452252,9.218898,2,Italian Restaurant,Vietnamese Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant
52,Bande Nere,45.459914,9.139080,2,Italian Restaurant,Café,Creperie,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant
78,Rogoredo - Santa Giulia,45.436672,9.243522,2,Italian Restaurant,Café,Creperie,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant


In [ ]:
milan_merged.loc[milan_merged['Cluster Labels'] == 3, milan_merged.columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Porta Garibaldi - Porta Nuova,45.483591,9.190579,3,Bakery,Wings Joint,Food,Dim Sum Restaurant,Diner,Doner Restaurant,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant


In [ ]:
milan_merged.loc[milan_merged['Cluster Labels'] == 4, milan_merged.columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ronchetto Sul Naviglio - Q.Re Lodovico Il Moro,45.438460,9.137260,4,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Restaurant,Bakery,Sushi Restaurant,Chinese Restaurant,Mediterranean Restaurant,Steakhouse
1,Trenno,45.492822,9.101675,4,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Burger Joint,Asian Restaurant
2,Portello,45.484490,9.153947,4,Italian Restaurant,Pizza Place,Japanese Restaurant,Café,Restaurant,Sushi Restaurant,Sandwich Place,Bakery,Bistro,Seafood Restaurant
3,Bovisasca,45.517433,9.156731,4,Italian Restaurant,Pizza Place,Café,Bakery,Diner,Restaurant,Japanese Restaurant,Kebab Restaurant,Chinese Restaurant,Doner Restaurant
5,Figino,45.491381,9.074376,4,Italian Restaurant,Pizza Place,Restaurant,Café,Fast Food Restaurant,Seafood Restaurant,Bistro,Breakfast Spot,Cafeteria,Emilia Restaurant
6,Quarto Oggiaro - Vialba - Musocco,45.513636,9.137731,4,Café,Italian Restaurant,Pizza Place,Fast Food Restaurant,Restaurant,Japanese Restaurant,BBQ Joint,Sushi Restaurant,Steakhouse,Piadineria
7,Isola,45.490894,9.189617,4,Italian Restaurant,Pizza Place,Seafood Restaurant,Japanese Restaurant,Chinese Restaurant,Restaurant,Puglia Restaurant,Café,Indian Restaurant,Bistro
9,Stadio - Ippodromi,45.479641,9.123833,4,Italian Restaurant,Sandwich Place,Pizza Place,Asian Restaurant,Bistro,Café,Restaurant,Chinese Restaurant,Sushi Restaurant,Steakhouse
11,Duomo,45.463707,9.186948,4,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Café,Pizza Place,Bakery,Restaurant,Argentinian Restaurant,Breakfast Spot,Bistro
12,Guastalla,45.463219,9.201891,4,Italian Restaurant,Japanese Restaurant,Restaurant,Café,Spanish Restaurant,Sandwich Place,Bakery,Brazilian Restaurant,Indian Restaurant,Diner
